In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import os
import sys
root_folders = ['..', '../..', './']
root_folders = [os.path.abspath(os.path.join(x)) for x in root_folders]
for path in root_folders:
    if path not in sys.path:
        sys.path.append(path)
# import nltk
%load_ext autoreload
%autoreload 2

In [ ]:
encoder = WordVocabEncoder.load('anki_index.json')

In [ ]:
text = 'Tom likes Mary, but Mary likes John. To make matters even more interesting, John likes Alice, but Alice likes Tom.'

In [ ]:
indices = encoder.to_indices(text)

In [ ]:
indices

In [ ]:
encoder.to_text(indices)

In [ ]:
pad_indices = encoder.pad_indices(indices)

In [ ]:
pad_indices

In [ ]:
encoder.to_text(pad_indices)

Из текста в последовательности с padding:

In [ ]:
encoder.text_to_indices(text)

Пример с генератором

In [ ]:
def generator():
    with open('ru_anki_corpora.txt', 'r', encoding='utf-8') as f, open('eng_anki_corpora.txt', 'r', encoding='utf-8') as d:
        ru_line, eng_line = f.readline().strip(), d.readline().strip()
        while ru_line:
            yield encoder.text_to_indices(eng_line, True), encoder.text_to_indices(ru_line,False)
            ru_line, eng_line = f.readline().strip(), d.readline().strip()

In [ ]:
dataset = tf.data.Dataset.from_generator(generator=generator,
                                         output_types=(tf.int64, tf.int64))

In [ ]:
dataset = dataset.batch(2, drop_remainder=True)

In [ ]:
for (x,y) in dataset:
    x, y = x.numpy(), y.numpy()
    x = [encoder.to_text(seq) for seq in x]
    y = [encoder.to_text(seq, is_source=False) for seq in y]
    for ru, eng in zip(x,y):
        print(ru, eng)
    break

In [ ]:
tf.__version__

In [6]:
from deeplavrov.models.wordlevel import WordLevelRNNTranslator

In [7]:
params = {
    'max_source_len' : 49,
    'max_target_len' : 51
}

In [8]:
translator = WordLevelRNNTranslator(**params)

In [9]:
translator.fit_from_file('./data/eng_anki_corpora.txt', './data/ru_anki_corpora.txt')

TypeError: `generator` must be callable.